In [ ]:

import pandas as pd
import datetime as datetime
from datetime import date
import numpy as np
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd

import numpy as np
from datetime import datetime
from datetime import datetime, timedelta


In [ ]:
df_merged_all = pd.read_csv('df_hutchison_4h.csv', sep='\t')


In [ ]:
df_merged_all = pd.read_csv('df_overnight_and_PRO.csv', sep='\t')


In [ ]:
df_merged_all['time'] = pd.to_datetime(df_merged_all['time'] ,errors = 'coerce')

df_merged_all = df_merged_all.reset_index(drop=True)
df_merged_all = df_merged_all.drop(["Unnamed: 0", "joinID"], axis=1)

df_merged_all = df_merged_all.rename(columns={'label': 'fasting_state'})


fasting_states_to_keep = ['fasting', 'non-fasting']
df_merged_all = df_merged_all[df_merged_all.fasting_state.isin(fasting_states_to_keep)]

final_df = df_merged_all.sort_values(by=['id', 'time'], ascending=True)
final_df['Day'] = final_df["time"].dt.date

## Olga Anfrage

In [ ]:
hutch_df = final_df

In [ ]:
map_dict = {"non-fasting": 1, "fasting":0}
hutch_df["fasting_state"] = hutch_df["fasting_state"].map(map_dict)


In [ ]:
hutch_df['temp'] = hutch_df['fasting_state'].diff()

In [ ]:
hutch_df['temp'] = hutch_df['temp'].replace({'0':np.nan, 0:np.nan})

In [ ]:
counter_zero = 1
counter_one = 0
last_phase = "screening"

hutch_df["series"] = np.nan

for index, row in hutch_df.iterrows():
    
    if last_phase == row['phase']:
    
        if  not np.isnan(row['temp']):

            if row['fasting_state'] == 0:
                counter_zero += 2
                hutch_df.at[index,'series'] = counter_zero


            if row['fasting_state'] == 1:
                counter_one += 2
                hutch_df.at[index,'series'] = counter_one

        elif np.isnan(row['temp']):

            if row['fasting_state'] == 0:
                hutch_df.at[index,'series'] = counter_zero

            if row['fasting_state'] == 1:
                hutch_df.at[index,'series'] = counter_one
    else:
            
            if row['fasting_state'] == 0:
                counter_zero += 2
                hutch_df.at[index,'series'] = counter_zero


            if row['fasting_state'] == 1:
                counter_one += 2
                hutch_df.at[index,'series'] = counter_one
                
            last_phase = row['phase']

In [ ]:
hutch_result_df = hutch_df.groupby(["phase", "id","series", "fasting_state"]).agg({'gl': 'mean', 'Day': lambda x: list(set(x))})

In [ ]:
hutch_result_df = hutch_df.groupby(["phase", "id","series", "fasting_state"]).agg({'gl': 'mean'})

In [ ]:
hutch_result_df = hutch_result_df.reset_index()
hutch_result_df = hutch_result_df.drop('series', axis=1)
hutch_result_df.rename(columns={'gl': 'gl_mean'}, inplace=True)
hutch_result_df = hutch_result_df.sort_values(by=['id', 'phase'], ascending=True)

In [ ]:
df_fastingGlucose = hutch_result_df.loc[hutch_result_df['fasting_state'] == 0]
df_postprandialGlucose = hutch_result_df.loc[hutch_result_df['fasting_state'] == 1]

In [ ]:
hutch_mean_fasting_glucose_per_phase = hutch_result_df.groupby(["id", "phase", "fasting_state"])["gl_mean"].mean().reset_index(name ='gl_mean_per_phase')


In [ ]:
df_meanFastingGlucosePerPhase = hutch_mean_fasting_glucose_per_phase.loc[hutch_mean_fasting_glucose_per_phase['fasting_state'] == 0]
df_meanPostprandialGlucosePerPhase = hutch_mean_fasting_glucose_per_phase.loc[hutch_mean_fasting_glucose_per_phase['fasting_state'] == 1]

In [ ]:
hutch_mean_fasting_glucose_per_phase

In [ ]:
writer = pd.ExcelWriter('hutchisonChronoFast.xlsx', engine='xlsxwriter')

In [ ]:
df_fastingGlucose.to_excel(writer, sheet_name='fastingGlucose')
df_meanFastingGlucosePerPhase.to_excel(writer, sheet_name='meanFastingGlucosePerPhase')
df_postprandialGlucose.to_excel(writer, sheet_name='postprandialGlucose')
df_meanPostprandialGlucosePerPhase.to_excel(writer, sheet_name='meanPostprandialGlucosePerPhase')

writer.save()

# für jeden Participant, die mean fasting glucose berechnen



In [ ]:
df_screening = final_df[(final_df['phase'] == "screening")]
df_eTRE = final_df[(final_df['phase'] == "eTRE")]
df_lTRE = final_df[(final_df['phase'] == "lTRE")]

In [ ]:
mean_fasting_glucose_per_phase_day = final_df.groupby(["id", "phase", "fasting_state", "Day"])["gl"].mean().reset_index(name ='gl_mean')
mean_fasting_glucose_per_phase = final_df.groupby(["id", "phase", "fasting_state"])["gl"].mean().reset_index(name ='gl_mean')


In [ ]:
mean_non_fasting_glucose= mean_fasting_glucose_per_phase[mean_fasting_glucose_per_phase.fasting_state != "non-fasting"]

In [ ]:
mean_non_fasting_glucose = mean_non_fasting_glucose.drop(["fasting_state"], axis=1)

In [ ]:
merged_df = pd.merge(final_df, mean_non_fasting_glucose,  on=["id", "phase"])

In [ ]:
def compliance_score(x): 
    
    if x['gl'] >= x['gl_mean'] and x['fasting_state'] == "non-fasting":
        return "compliant"
    elif x['gl'] > x['gl_mean'] and x['fasting_state'] == "fasting":
        return "non-compliant"    
    elif x['gl'] <= x['gl_mean'] and x['fasting_state'] == "non-fasting":
        return "non-compliant"
    elif x['gl'] < x['gl_mean'] and x['fasting_state'] == "fasting":
        return "compliant"
   

In [ ]:
merged_df["compliance_score"] = merged_df.apply(compliance_score, axis=1)

In [ ]:
merged_df_eTRE = merged_df[(merged_df['phase'] == "eTRE")]
merged_df_without_screening = merged_df[(merged_df['phase'] != "screening")]

merged_df_lTRE = merged_df[(merged_df['phase'] == "lTRE")]

In [ ]:
merged_df

In [ ]:
result_lTRE = merged_df_lTRE.compliance_score.value_counts()

In [ ]:
compliance_score_lTRE = result_lTRE[0] / (result_lTRE[0]+ result_lTRE[1])*100

In [ ]:
compliance_score_lTRE

In [ ]:
result_eTRELTRE = merged_df_without_screening.compliance_score.value_counts()

In [ ]:
result_eTRELTRE

In [ ]:
compliance_score_eTRELTRE = result_eTRELTRE[0] / (result_eTRELTRE[0]+ result_eTRELTRE[1])*100

In [ ]:
compliance_score_eTRELTRE

In [ ]:
result = merged_df.compliance_score.value_counts()

In [ ]:
compliance_score = result[0] / (result[0]+ result[1])*100

In [ ]:
compliance_score

### Accuracy hutchison all


In [ ]:
merged_df["fasting_state_pred"] = merged_df.apply(make_prediction_hutchison, axis=1)


In [ ]:
result_all = merged_df.compliance_score.value_counts()

In [ ]:
compliance_score_all = result_all[0] / (result_all[0]+ result_all[1])*100

In [ ]:
compliance_score_all

In [ ]:
from numpy.lib.stride_tricks import sliding_window_view


def smoothing(y_pred):
    
    outcomes = np.array([])
    last_four_values = y_pred[-4:]
    
    y_pred_windowed = sliding_window_view((y_pred), window_shape = 5)

    i = 0
    for window in y_pred_windowed:
        pred_one = np.count_nonzero(window == 1)
        pred_zero = 5 - pred_one
        if pred_one > pred_zero:
            outcome = int(1)
            outcomes = np.append(outcomes, outcome)
        else:
            outcome = int(0)
            outcomes = np.append(outcomes, outcome)
    
    outcomes = np.append(outcomes, last_four_values)
            
    
    return outcomes

In [ ]:
map_dict = {"non-fasting": 1, "fasting":0}
merged_df["fasting_state_pred"] = merged_df["fasting_state_pred"].map(map_dict)


In [ ]:
merged_df["fasting_state"] = merged_df["fasting_state"].map(map_dict)


In [ ]:
result_all_np = merged_df["fasting_state_pred"]
result_actual = merged_df["fasting_state"]

In [ ]:
result_all_np = np.array(result_all_np)
result_actual = np.array(result_actual)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(result_actual, result_all_np)

In [ ]:
smoothed_score = smoothing(result_all_np)

In [ ]:
accuracy_score(result_actual, smoothed_score)

In [ ]:
merged_df["pred_fasting_state"] =  merged_df.apply(make_prediction_hutchison, axis=1)

In [ ]:
mask= ["id", "time", "phase", "gl", "gl_mean", "fasting_state", "pred_fasting_state", "compliance_score"]

In [ ]:
merged_df[mask]